In [0]:
# %sql
# CREATE DATABASE IF NOT EXISTS poc_db;

In [0]:
# %sql
# DESCRIBE POC_DB.taxi_nyc;

In [0]:
# %sql
# CREATE TABLE poc_db.nyc_taxi_raw
# USING PARQUET
# LOCATION 'wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow';

In [0]:
# display(dbutils.fs.ls('/FileStore/tables/nyctaxi/'))

In [0]:
# import zipfile

# zip_path = "/dbfs/FileStore/tables/nyctaxi/NYC_Yellow_Taxi_Trip_Data.zip"
# extract_path = "/dbfs/FileStore/tables/nyctaxi/unzipped"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print("Unzipped successfully")

In [0]:
# display(dbutils.fs.ls("/FileStore/tables/nyctaxi/unzipped"))

In [0]:
# from pyspark.sql.types import *

# schema = StructType([

#     StructField("VendorID", IntegerType(), True),

#     StructField("tpep_pickup_datetime", TimestampType(), True),
#     StructField("tpep_dropoff_datetime", TimestampType(), True),

#     StructField("passenger_count", IntegerType(), True),
#     StructField("trip_distance", DoubleType(), True),

#     StructField("pickup_longitude", DoubleType(), True),
#     StructField("pickup_latitude", DoubleType(), True),

#     StructField("RateCodeID", IntegerType(), True),
#     StructField("store_and_fwd_flag", StringType(), True),

#     StructField("dropoff_longitude", DoubleType(), True),
#     StructField("dropoff_latitude", DoubleType(), True),

#     StructField("payment_type", IntegerType(), True),

#     StructField("fare_amount", DoubleType(), True),
#     StructField("extra", DoubleType(), True),
#     StructField("mta_tax", DoubleType(), True),

#     StructField("tip_amount", DoubleType(), True),
#     StructField("tolls_amount", DoubleType(), True),
#     StructField("improvement_surcharge", DoubleType(), True),

#     StructField("total_amount", DoubleType(), True),
# ])


In [0]:
# df = (
#     spark.read
#     .schema(schema)
#     .option("header", "true")
#     .option("timestampFormat", "yyyy-MM-dd HH:mm:ss")
#     .csv("/FileStore/tables/nyctaxi/unzipped/*.csv")
# )

In [0]:
# df.write \
#   .format("delta") \
#   .mode("overwrite") \
#   .saveAsTable("poc_db.nyc_taxi_raw")

In [0]:
# %sql
# CREATE OR REPLACE TABLE poc_db.bi_taxi AS
# SELECT
#     YEAR(tpep_pickup_datetime)  AS year,
#     MONTH(tpep_pickup_datetime) AS month,
#     DAY(tpep_pickup_datetime)   AS day,

#     payment_type,
#     vendorid AS vendor_id,

#     CASE
#         WHEN HOUR(tpep_pickup_datetime) BETWEEN 5  AND 11 THEN 'Morning'
#         WHEN HOUR(tpep_pickup_datetime) BETWEEN 12 AND 16 THEN 'Afternoon'
#         WHEN HOUR(tpep_pickup_datetime) BETWEEN 17 AND 21 THEN 'Evening'
#         ELSE 'Night'
#     END AS time_bucket,

#     passenger_count,
#     trip_distance,
#     fare_amount,
#     tip_amount,
#     tolls_amount,
#     total_amount

# FROM poc_db.nyc_taxi_raw;


In [0]:
%sql
CREATE OR REPLACE TABLE poc_db.bi_taxi AS
SELECT
    -- Date parts
    YEAR(tpep_pickup_datetime)  AS year,
    MONTH(tpep_pickup_datetime) AS month,
    DAY(tpep_pickup_datetime)   AS day,

    -- Dimensions
    payment_type,
    vendorid AS vendor_id,

    CASE
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 5  AND 11 THEN 'Morning'
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 17 AND 21 THEN 'Evening'
        ELSE 'Night'
    END AS time_bucket,

    -- Trip metrics
    passenger_count,
    trip_distance,

    -- 💰 All monetary fields
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    improvement_surcharge,
    total_amount

FROM poc_db.nyc_taxi_raw;


In [0]:
%sql
OPTIMIZE poc_db.bi_taxi;

In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,

  -- Distance
  SUM(CASE WHEN trip_distance <= 0 OR trip_distance > 50 THEN 1 ELSE 0 END) AS bad_distance,

  -- Fare
  SUM(CASE WHEN fare_amount < 2.5 OR fare_amount > 500 THEN 1 ELSE 0 END) AS bad_fare,

  -- Total vs Fare
  SUM(CASE WHEN total_amount < fare_amount THEN 1 ELSE 0 END) AS bad_total_logic,

  -- Passengers
  SUM(CASE WHEN passenger_count <= 0 OR passenger_count > 6 THEN 1 ELSE 0 END) AS bad_passengers,

  -- Tips
  SUM(CASE WHEN tip_amount < 0 OR tip_amount > 200 THEN 1 ELSE 0 END) AS bad_tips

FROM poc_db.bi_taxi;


In [0]:
%sql
DELETE FROM poc_db.bi_taxi
WHERE
    trip_distance NOT BETWEEN 0.1 AND 50
    OR fare_amount NOT BETWEEN 2.5 AND 500
    OR total_amount < fare_amount
    OR passenger_count NOT BETWEEN 1 AND 6
    OR tip_amount NOT BETWEEN 0 AND 200;


In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,
  MIN(trip_distance), MAX(trip_distance),
  MIN(fare_amount), MAX(fare_amount),
  MIN(total_amount), MAX(total_amount),
  MIN(passenger_count), MAX(passenger_count),
  MIN(tip_amount), MAX(tip_amount)
FROM poc_db.bi_taxi;



In [0]:
%sql
describe table poc_db.bi_taxi

In [0]:
%sql
SELECT DISTINCT time_bucket
FROM poc_db.bi_taxi;


In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,

  MIN(trip_distance), MAX(trip_distance),
  MIN(fare_amount), MAX(fare_amount),
  MIN(total_amount), MAX(total_amount),
  MIN(passenger_count), MAX(passenger_count),
  MIN(tip_amount), MAX(tip_amount),

  MIN(extra), MAX(extra),
  MIN(mta_tax), MAX(mta_tax),
  MIN(tolls_amount), MAX(tolls_amount),
  MIN(improvement_surcharge), MAX(improvement_surcharge),

  MIN(year), MAX(year),
  MIN(month), MAX(month),
  MIN(day), MAX(day)

FROM poc_db.bi_taxi;


In [0]:
%sql
DELETE FROM poc_db.bi_taxi
WHERE
    extra NOT BETWEEN 0 AND 20
    OR mta_tax NOT BETWEEN 0 AND 1
    OR tolls_amount NOT BETWEEN 0 AND 200;
